In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Normalization
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

2023-01-06 23:27:45.179901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 23:27:45.347041: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-06 23:27:45.347056: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-06 23:27:46.075115: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [18]:
def make_model(model_shape):
    # print(model_shape)
    # print(model_shape.shape[1])
    # normal = Normalization(input_shape=[model_shape.shape[1],], axis=None)
    # normal.adapt(model_shape)
    model = Sequential()
    # model.add(normal)
    # model.add(Dense(units=4, activation="relu"))
    # model.add(Dense(units=8, activation="relu"))
    # model.add(Dense(units=16, activation="relu"))
    # model.add(Dense(units=32, activation="relu"))
    # model.add(Dense(units=64, activation="relu"))
    # model.add(Dense(units=128, activation="relu"))
    model.add(Dense(units=256, activation="relu", input_shape=[model_shape.shape[1],]))
    model.add(Dense(units=256, activation="relu"))
    model.add(Dense(units=256, activation="relu"))
    model.add(Dense(units=128, activation="relu"))
    model.add(Dense(units=128, activation="relu"))
    model.add(Dense(units=128, activation="relu"))
    model.add(Dense(units=64, activation="relu"))
    model.add(Dense(units=64, activation="relu"))
    model.add(Dense(units=64, activation="relu"))
    model.add(Dense(units=1, activation="relu"))
    # model.add(Dense(units=1000000, activation="relu"))
    # model.add(Dense(units=10, activation="relu"))
    # RED WINE
    # model.add(Dense(units=4, input_shape=(model_shape.shape[1],), activation="relu"))
    # model.add(Dense(units=8, activation="relu"))
    # model.add(Dense(units=16, activation="relu"))
    # model.add(Dense(units=32, activation="relu"))
    # model.add(Dense(units=64, activation="relu"))
    # model.add(Dense(units=128, activation="relu"))
    # model.add(Dense(units=98, activation="relu"))
    # model.add(Dense(units=46, activation="relu"))
    # model.add(Dense(units=32, activation="relu"))
    # model.add(Dense(units=12, activation="relu"))
    # model.add(Dense(units=10, activation="relu"))
    model.compile(optimizer="adam", metrics=['mae', 'mse'], loss='mse')
    #model.compile(optimizer="adam", metrics=['accuracy'], loss='binary_crossentropy')
    model.summary()
    return model

In [4]:
def load_and_purge_data():
    raw_data = pd.read_csv("vehicles.csv")
    #picked_columns = raw_data[["price", "year", "manufacturer", "model", "odometer", "drive"]]
    picked_columns = raw_data[["price", "year", "odometer"]]
    data_without_nulls = picked_columns.dropna(axis=0)
    print(data_without_nulls.isnull().sum())
    return data_without_nulls

In [20]:
def split_data(data, valid_col):
    size = len(data.index)
    p7 = int((70 * size) / 100)
    p2 = int((20 * size) / 100)
    p1 = int((10 * size) / 100)
    v_col = data[[valid_col]]
    data = data.drop(valid_col, axis=1)
    part1 = data[1 : p7]
    part2 = data[p7 : p7+p2]
    part3 = data[p7+p2 : p7+p2+p1]
    part1_v_col = v_col[1 : p7]
    part2_v_col = v_col[p7 : p7+p2]
    part3_v_col = v_col[p7+p2 : p7+p2+p1]
    return part1, part2, part3, part1_v_col, part2_v_col, part3_v_col

In [21]:
def data_splitter(data_x, data_y):
    return train_test_split(data_x, data_y, test_size=0.3, random_state=101)

In [6]:
def red_wine_load():
    wine_raw_data = pd.read_csv("winequality-red.csv")
    rw_x = wine_raw_data.drop('quality', axis=1)
    rw_y = wine_raw_data['quality']
    rw_x_train, rw_x_test, rw_y_train, rw_y_test = train_test_split(rw_x, rw_y, test_size=0.3, random_state=101)
    return rw_x_train, rw_x_test, rw_y_train, rw_y_test

In [ ]:
# RED WINE
# w_x_train, w_x_test, w_y_train, w_y_test = red_wine_load()
# w_r_data = pd.read_csv("winequality-red.csv")
# w_x_train, w_x_test, w_x_val, w_y_train, w_y_test, w_y_val = split_data(w_r_data, "quality")
# mdl = make_model(w_x_train)
# history = mdl.fit(w_x_train, w_y_train, batch_size=32, epochs=100, validation_data=(w_x_test, w_y_test))
# results = mdl.evaluate(w_x_val, w_y_val, batch_size=128)
# print("test loss, test acc:", results)

# CARS
# x_train, x_test, x_val, y_train, y_test, y_val = split_data(load_and_purge_data(), "price")
cars_f = load_and_purge_data()
val_col = cars_f[["price"]]
cars_f = cars_f.drop("price", axis=1)
x_train, x_test, y_train, y_test = data_splitter(cars_f, val_col)
mdl = make_model(x_train)
history = mdl.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))
# results = mdl.evaluate(x_val, y_val, batch_size=128)
print("test loss, test acc:", history)